Find top 1 outlets by cuisine type without using limit and top function

In [1]:
WITH cte AS (
    SELECT 
        Cuisine, 
        Restaurant_id, 
        COUNT(*) AS no_of_order 
    FROM data_restro.csv
    GROUP BY Cuisine, Restaurant_id
)
SELECT * 
FROM (
    SELECT 
        *, 
        ROW_NUMBER() OVER (PARTITION BY Cuisine ORDER BY no_of_order DESC) AS rn 
    FROM cte
) AS a

where rn =1;


,Cuisine,Restaurant_id,no_of_order,rn
0,Mexican,TACO789,7,1
1,Lebanese,KMKMH6787,10,1
2,Japanese,SUSHI456,6,1
3,American,BURGER99,8,1
4,Italian,PIZZA123,10,1


Find the daily new customer count from the launch date (everyday how many new customers are we acquiring)

In [2]:
WITH cte AS (
    SELECT 
        Customer_code, 
        CAST(MIN(placed_at) AS DATE) AS first_order_date
    FROM data_restro.csv
    GROUP BY Customer_code
)
SELECT 
    first_order_date, 
    COUNT(*) AS no_of_new_customers
FROM cte
GROUP BY first_order_date
ORDER BY first_order_date;


,first_order_date,no_of_new_customers
0,2025-01-01 00:00:00+00:00,2
1,2025-01-02 00:00:00+00:00,1
2,2025-01-03 00:00:00+00:00,1
3,2025-01-04 00:00:00+00:00,1
4,2025-01-05 00:00:00+00:00,3
5,2025-01-06 00:00:00+00:00,1
6,2025-01-07 00:00:00+00:00,1
7,2025-01-08 00:00:00+00:00,1
8,2025-01-09 00:00:00+00:00,1
9,2025-01-10 00:00:00+00:00,3


3 Count of all the users who were acquired in Jan 2025 and only placed one order in Jan and did lot place any other order.

In [1]:
SELECT Customer_code, COUNT(*) AS no_of_orders
FROM data_restro.csv
WHERE MONTH(placed_at) = 1 
  AND YEAR(placed_at) = 2025
GROUP BY Customer_code
HAVING COUNT(*) = 1
   AND Customer_code NOT IN (
       SELECT Customer_code
       FROM data_restro.csv
       WHERE NOT (MONTH(placed_at) = 1 AND YEAR(placed_at) = 2025)
   );


,Customer_code,no_of_orders
0,GHI5678901234XYZ,1
1,STU9876543210ABC,1
2,HIJ9876543210DEF,1
3,NOP3456789012DEF,1
4,QRS7890123456DEF,1
5,WXY9876543210GHI,1
6,LMN9876543210JKL,1
7,DEF5678901234MNO,1
8,PQR9876543210PQR,1
9,STU5678901234PQR,1


List All the customers with no order in the last 7 days but were acquired one month ago with their first order on promo.

In [2]:
WITH cte AS (
    SELECT 
        Customer_code, 
        MIN(placed_at) AS first_order_date, 
        MAX(placed_at) AS latest_order_date
    FROM Orders
    GROUP BY Customer_code
)

SELECT 
    cte.*, 
    o.Promo_code_Name AS first_order_promo
FROM cte
INNER JOIN Orders o 
    ON cte.Customer_code = o.Customer_code 
    AND cte.first_order_date = o.placed_at
WHERE 
    cte.latest_order_date < DATEADD(DAY, -7, GETDATE()) AND
    cte.first_order_date < DATEADD(MONTH, -1, GETDATE()) AND
    o.Promo_code_Name IS NOT NULL;


,Customer_code,first_order_date,latest_order_date
0,UFDDN1991918XUY1,2025-01-01 15:30:20+00:00,2025-03-28 11:30:00+00:00
1,DEF9876543210XYZ,2025-01-02 09:15:00+00:00,2025-03-02 09:15:00+00:00
2,GHI5678901234XYZ,2025-01-03 14:30:00+00:00,2025-01-03 14:30:00+00:00
3,STU9876543210ABC,2025-01-07 13:15:00+00:00,2025-01-07 13:15:00+00:00
4,HIJ9876543210DEF,2025-01-12 14:45:00+00:00,2025-01-12 14:45:00+00:00
5,NOP3456789012DEF,2025-01-14 12:15:00+00:00,2025-01-14 12:15:00+00:00
6,QRS7890123456DEF,2025-01-15 19:00:00+00:00,2025-01-15 19:00:00+00:00
7,WXY9876543210GHI,2025-01-17 15:30:00+00:00,2025-01-17 15:30:00+00:00
8,LMN9876543210JKL,2025-01-22 14:30:00+00:00,2025-01-22 14:30:00+00:00
9,UVW7890123456JKL,2025-01-25 19:15:00+00:00,2025-03-25 19:15:00+00:00
